In [ ]:
# ------------------------------------------
# Was tut dieses Skript?
# Dieses Skript verarbeitet Mapillary-Coverage-Daten für OSM-Highways in Deutschland.
# Es liest die Coverage-Daten für "pano" und "regular", filtert sie nach einem Coverage-Ratio >= 0.6,
# kombiniert die Ergebnisse, entfernt Duplikate (bevorzugt "pano"), und speichert die finale Tabelle als CSV.
# siehe https://github.com/vizsim/mapillary_coverage/issues/1
# ------------------------------------------

In [1]:
import pandas as pd
import geopandas as gpd
# Read the Parquet files


#gdf_mp_pano = pd.read_parquet("germany_osm-highways_25-04-05_mp_pano_coverage_23-01-01until25-04-01_ratio.parquet")
#gdf_mp_all = pd.read_parquet("germany_osm-highways_25-04-05_mp_all_coverage_23-01-01until25-04-01_ratio.parquet")

# gdf_mp_pano = pd.read_parquet("germany_osm-highways_25-05-09_mp_pano_coverage_25-05-10_ratio.parquet")
# gdf_mp_regular = pd.read_parquet("germany_osm-highways_25-05-09_mp_regular_coverage_25-05-10_ratio.parquet")


gdf_mp_pano = pd.read_parquet("germany_osm-highways_25-10-06_mp_pano_coverage_25-10-07_ratio.parquet")
gdf_mp_regular = pd.read_parquet("germany_osm-highways_25-10-06_mp_regular_coverage_25-10-07_ratio.parquet")


In [2]:
gdf_mp_pano=gdf_mp_pano[["osm_id","mp_coverage_ratio"]].copy()

In [3]:
gdf_mp_pano_above_06=gdf_mp_pano[gdf_mp_pano["mp_coverage_ratio"]>=0.6].copy()

In [4]:
gdf_mp_pano_above_06["mapillary_coverage"] = "pano"

In [5]:
gdf_mp_pano_above_06

,osm_id,mp_coverage_ratio,mapillary_coverage
0,680,1.0,pano
1,1978,1.0,pano
5,2293021,1.0,pano
7,2413197,1.0,pano
8,2413202,1.0,pano
...,...,...,...
764150,1435981806,1.0,pano
764151,1435981807,1.0,pano
764152,1435981808,1.0,pano
764153,1435981809,1.0,pano


In [6]:
gdf_mp_regular=gdf_mp_regular[["osm_id","mp_coverage_ratio"]].copy()

In [7]:
gdf_mp_regular_above_06=gdf_mp_regular[gdf_mp_regular["mp_coverage_ratio"]>=0.6].copy()

In [8]:
gdf_mp_regular_above_06

,osm_id,mp_coverage_ratio
2,104,1.000000
6,111,1.000000
7,117,1.000000
8,122,1.000000
9,123,0.868701
...,...,...
2983013,1435994704,1.000000
2983014,1435994705,1.000000
2983015,1435994706,1.000000
2983016,1435994707,1.000000


In [9]:
gdf_mp_regular_above_06["mapillary_coverage"] = "regular"

In [10]:
both_concat=pd.concat([gdf_mp_pano_above_06,gdf_mp_regular_above_06],ignore_index=True)

In [11]:
both_concat["mapillary_coverage"].value_counts()

mapillary_coverage
regular    1631743
pano        427675
Name: count, dtype: int64

In [13]:
## drop duplicates, keep pano
both_concat = both_concat.sort_values(by="mapillary_coverage", ascending=True).drop_duplicates(subset="osm_id", keep="first")

In [14]:
both_concat["mapillary_coverage"].value_counts()

mapillary_coverage
regular    1450647
pano        427675
Name: count, dtype: int64

In [15]:
both_concat["osm_id"].value_counts()

osm_id
680           1
1062878716    1
1062878715    1
1062878115    1
1062828053    1
             ..
187549939     1
187548520     1
187548519     1
187545910     1
1435994708    1
Name: count, Length: 1878322, dtype: int64

In [16]:
both_concat=both_concat[["osm_id","mapillary_coverage"]].copy()

In [17]:
both_concat

,osm_id,mapillary_coverage
0,680,pano
285123,1062878716,pano
285122,1062878715,pano
285121,1062878115,pano
285120,1062828053,pano
...,...,...
971584,187549939,regular
971583,187548520,regular
971582,187548519,regular
971580,187545910,regular


In [18]:
both_concat.to_csv("output/germany_osm-highways_25-10-06_mp_coverage_23-01-01until25-10-07_ratio_above_06.csv",index=False)